In [7]:
import duckdb

%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [8]:
%%sql 
INSTALL spatial;
LOAD spatial;
INSTALL h3 FROM community;
LOAD h3;
INSTALL postgres;
LOAD postgres;
INSTALL sqlite;
LOAD sqlite;


Running query in 'duckdb'

Success


- Need the Postgres.app to be running here

In [9]:
%%sql
CREATE OR REPLACE TABLE cities AS
SELECT id as efua_id, name, fua_pop, cnt_iso FROM postgres_scan('host=localhost port=5432 user=jonathan dbname=territoires.fyi', 'dbt_urbanized_fyi', 'cities');

Running query in 'duckdb'

Count
2000


In [10]:
%sql SELECT * FROM cities LIMIT 5;

Running query in 'duckdb'

efua_id,name,fua_pop,cnt_iso
9725,Guangzhou,45640589,['CHN']
4900,Jakarta,39808169,['IDN']
5129,Tokyo,36471787,['JPN']
7466,Delhi [New Delhi],30076708,['IND']
10097,Shanghai,26920218,['CHN']


In [11]:
%%sql
CREATE OR REPLACE TABLE 
h3_efua_pop AS
SELECT * FROM postgres_scan('host=localhost port=5432 user=jonathan dbname=territoires.fyi', 'dbt_urbanized_fyi', 'efua_h3_pop');

Running query in 'duckdb'

Count
3601676


In [12]:
%%sql
CREATE OR REPLACE TABLE 
source_kontur AS
SELECT h3, population FROM ST_Read('../_input/kontur.gpkg');


Running query in 'duckdb'

Count
32957699


In [13]:
%%sql
CREATE OR REPLACE TABLE h3_efua_pop AS
SELECT h3_efua_pop.h3, h3_efua_pop.efua_id, source_kontur.population 
FROM h3_efua_pop 
JOIN source_kontur ON h3_efua_pop.h3 = source_kontur.h3;

Running query in 'duckdb'

Count
3086206


In [14]:
%%sql
CREATE OR REPLACE TABLE cities AS
SELECT cities.*, kontur_population.kontur_population AS kontur_population
FROM cities
JOIN (
  SELECT cities.efua_id, SUM(h3_efua_pop.population) AS kontur_population
  FROM cities
  JOIN h3_efua_pop ON cities.efua_id = h3_efua_pop.efua_id
  GROUP BY cities.efua_id
) AS kontur_population
ON cities.efua_id = kontur_population.efua_id;

Running query in 'duckdb'

Count
2000


In [15]:
%%sql
SELECT 
  efua_id,
  name,
  cnt_iso,
  CAST(fua_pop AS INTEGER) as fua_pop,
  CAST(kontur_population AS INTEGER) as kontur_pop
FROM cities LIMIT 5;

Running query in 'duckdb'

efua_id,name,cnt_iso,fua_pop,kontur_pop
8353,Jhansi,['IND'],698987,467864
8407,Lalitpur,['IND'],481590,127850
8303,Datia,['IND'],358537,118807
6677,Shakar Garh,['PAK'],387206,175949
9684,Mau,['IND'],718010,289778


In [16]:
%%sql
COPY (
  SELECT 
    efua_id,
    name,
    cnt_iso,
    CAST(fua_pop AS INTEGER) as fua_pop,
    CAST(kontur_population AS INTEGER) as kontur_pop
  FROM cities
) TO '../_output/notebooks/cities.csv' (HEADER, DELIMITER ',');
COPY (
  SELECT
    h3,
    efua_id,
    CAST(population AS INTEGER) as population
  FROM h3_efua_pop
) TO '../_output/notebooks/cities_h3_pop.csv' (HEADER, DELIMITER ',');

Running query in 'duckdb'

Count
3086206


In [19]:
%%sql
# ATTACH '../_output/notebooks/cities.db' AS sqlite_db (TYPE SQLITE);
CREATE TABLE sqlite_db.cities AS (
    SELECT 
    efua_id,
    name,
    cnt_iso,
    CAST(fua_pop AS INTEGER) as fua_pop,
    CAST(kontur_population AS INTEGER) as kontur_pop
  FROM cities
);
CREATE TABLE sqlite_db.cities_h3_pop AS (
    SELECT
    h3,
    efua_id,
    CAST(population AS INTEGER) as population
  FROM h3_efua_pop
);

Running query in 'duckdb'

Count
3086206


## Notes
- The sqlite dump above will have a few lines that Cloudflare D1 does not like

At the top:
- `PRAGMA foreign_keys=OFF;`
- `BEGIN TRANSACTION;`

And at the bottom:
- `COMMIT;`

In [20]:
%%bash
pnpx wrangler d1 execute urbanized --remote --file .data/_source/cities.sql

cities.ipynb
